In [3]:
from func_def import *

In [4]:
events = get_links(url='http://www.ufcstats.com/statistics/events/completed?page=all')
events.date = pd.to_datetime(events.date)
events = events[events.date < get_current_date()].reset_index(drop=True)
events.to_csv('event_list.csv')
events

,date,event_url
0,2023-09-23,http://www.ufcstats.com/event-details/c945adc2...
1,2023-09-16,http://www.ufcstats.com/event-details/8fa2b065...
2,2023-09-09,http://www.ufcstats.com/event-details/ece28074...
3,2023-09-02,http://www.ufcstats.com/event-details/ef61d9f5...
4,2023-08-26,http://www.ufcstats.com/event-details/89a40703...
...,...,...
659,1995-07-14,http://www.ufcstats.com/event-details/1c3f5e85...
660,1995-04-07,http://www.ufcstats.com/event-details/dedc3bb4...
661,1994-12-16,http://www.ufcstats.com/event-details/b60391da...
662,1994-09-09,http://www.ufcstats.com/event-details/1a49e067...


In [11]:
#get existing data
UPDATE_ALL = True

exist_data = []
exist_events_url = []

if not UPDATE_ALL:
    try:
        exist_data = pd.read_csv('data_pre_processed.csv',index_col=0)
        exist_events_url = list(np.unique(exist_data.event_url))
    except:
        print('Cant find data_pre_processed.csv')
        exist_data = []
        exist_events_url = []

print(f'Current Existing events: {len(exist_events_url)}')

Current Existing events: 0


In [12]:
#get new data
new_events = events[~events.event_url.isin(exist_events_url)].reset_index(drop=True)
#new_events = events
print(f'New events: {len(new_events)}')

New events: 664


In [13]:
new_data = []
for url,time in tqdm(zip(new_events.event_url,new_events.date)):
    try:
        new_data.extend(get_event_data(EVENT_URL=url,EVENT_TIME=time))
    except:
        continue   

try:
    data = pd.concat([exist_data,pd.DataFrame(new_data)]).reset_index(drop=True)
except:
    data = new_data
data.tail(3)    

0it [00:00, ?it/s]

664it [3:45:49, 20.41s/it]


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [16]:
data = pd.DataFrame(new_data) 
data.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_sig_str,f_body_sig_str,f_leg_sig_str,f_distance_sig_str,f_clinch_sig_str,f_ground_sig_str,o_head_sig_str,o_body_sig_str,o_leg_sig_str,o_distance_sig_str,o_clinch_sig_str,o_ground_sig_str,f_sig_str_head_perc,f_sig_str_body_perc,f_sig_str_leg_perc,f_sig_str_dist_perc,f_sig_str_clinch_perc,f_sig_str_ground_perc,o_sig_str_head_perc,o_sig_str_body_perc,o_sig_str_leg_perc,o_sig_str_dist_perc,o_sig_str_clinch_perc,o_sig_str_ground_perc
7192,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7193,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7194,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [19]:
pd.DataFrame(data).to_csv('data_pre_processed.csv',index=[0])